In [2]:
import pandas as pd
train_file_path = "mxm_dataset_train.txt"
test_file_path = "mxm_dataset_test.txt"

# Initialize lists to store the data
words_data = []
lyrics_data = []

# Helper function to encode strings like the SQLite utility
def encode_string(s):
    return s.replace("'", "''")

# Parse the training data file for words
with open(train_file_path, 'r', encoding='utf-8') as file:
    for line in file:
        if line.startswith('%'):
            topwords = line.strip()[1:].split(',')
            break

# Add the words to the words table (DataFrame)
words_df = pd.DataFrame(topwords, columns=["word"])
words_df

,word
0,i
1,the
2,you
3,to
4,and
...,...
4995,santo
4996,pe
4997,gee
4998,colleg


In [9]:
# words_df.insert(0, 'id', range(1, len(words_df) + 1))
words_df = words_df.rename(columns={'id':'word_id', 'word':'word'})

In [10]:
words_df.head()

,word_id,word
0,1,i
1,2,the
2,3,you
3,4,to
4,5,and


In [3]:
!pip install sqlalchemy

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 2.3 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 599.5/599.5 KB 2.3 MB/s eta 0:00:0000:01


In [3]:
import pandas as pd
from sqlalchemy import create_engine

# PostgreSQL connection details
db_user = "sparkuser"
db_password = "9567"
db_host = "localhost"  # Change if needed
db_port = "5432"  # Default PostgreSQL port
db_name = "lyricsdb"

# Create SQLAlchemy engine
engine = create_engine(f"postgresql+psycopg2://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}")

# Append data to an existing table
# words_df.to_sql("words", engine, if_exists="append", index=False)

In [4]:
# Parse the training data file for lyrics
train_file_path = "mxm_dataset_train.txt"
lyrics_data = []
with open(train_file_path, 'r', encoding='utf-8') as file:
    for line in file:
        if not line.strip() or line.startswith(('%', '#')):
            continue

        parts = line.strip().split(',')
        track_id, mxm_tid = parts[:2]

        # Split word and count pairs
        for word_count in parts[2:]:
            word, count = word_count.split(':')
            lyrics_data.append([track_id, mxm_tid, word, int(count), 0])  # is_test = 0 for training data

lyrics_df = pd.DataFrame(lyrics_data, columns=["track_id", "mxm_tid", "word", "count", "is_test"])
lyrics_df

,track_id,mxm_tid,word,count,is_test
0,TRAAAAV128F421A322,4623710,1,6,0
1,TRAAAAV128F421A322,4623710,2,4,0
2,TRAAAAV128F421A322,4623710,3,2,0
3,TRAAAAV128F421A322,4623710,4,2,0
4,TRAAAAV128F421A322,4623710,5,5,0
...,...,...,...,...,...
16845938,TRZZZYX128F92D32C6,681124,3355,1,0
16845939,TRZZZYX128F92D32C6,681124,4198,1,0
16845940,TRZZZYX128F92D32C6,681124,4356,1,0
16845941,TRZZZYX128F92D32C6,681124,4738,1,0


In [5]:
lyrics_df = lyrics_df.rename(columns={'word':'word_id'})
lyrics_df['is_test'] = lyrics_df['is_test'].astype(bool)
lyrics_df.insert(5, 'l_id', range(1, len(lyrics_df) + 1))
lyrics_df.head()

,track_id,mxm_tid,word_id,count,is_test,l_id
0,TRAAAAV128F421A322,4623710,1,6,False,1
1,TRAAAAV128F421A322,4623710,2,4,False,2
2,TRAAAAV128F421A322,4623710,3,2,False,3
3,TRAAAAV128F421A322,4623710,4,2,False,4
4,TRAAAAV128F421A322,4623710,5,5,False,5


In [8]:
# lyrics_df[50001:100000].to_sql("lyrics", engine, if_exists="append", index=False)

In [ ]:
for i in range(100000, len(lyrics_df), 50000):
    if (i+50000) < len(lyrics_df):
        lyrics_df[i+1:i+50000].to_sql("lyrics", engine, if_exists="append", index=False)
    else:
        lyrics_df[i+1:].to_sql("lyrics", engine, if_exists="append", index=False)

    print(f"Rows from {i+1} to {i+50000} uploaded")

Rows from 100001 to 150000 uploaded
Rows from 150001 to 200000 uploaded
Rows from 200001 to 250000 uploaded
Rows from 250001 to 300000 uploaded
Rows from 300001 to 350000 uploaded
Rows from 350001 to 400000 uploaded
Rows from 400001 to 450000 uploaded
Rows from 450001 to 500000 uploaded
Rows from 500001 to 550000 uploaded
Rows from 550001 to 600000 uploaded
Rows from 600001 to 650000 uploaded
